## Flywheel model

$$
\left\{ \begin{align*}
         \frac{d\omega_r}{dt} &= \frac{1}{2H} \left(-\Phi_f i_q \right), \\		   
		\frac{dv_c}{dt} &= \frac{1}{C_{eq}} \left(i_g- \omega_r \Phi_f  \frac{i_q}{v_c}  \right)
\end{align*} \right.
		\qquad
$$

with:

$$
\left\{ \begin{align*}
         P_g &= P_{fes}, \\		   
		 i_g  &=   \frac{P_g}{v_c}, \\
         i_q  &=   \frac{P_g}{\omega_r \Phi_f}  
\end{align*} \right.
		\qquad
$$


In [41]:
import PyDSTool as dst
from PyDSTool import args
import numpy as np
from matplotlib import pyplot as plt

E_kwh = 250.0 # kWh
P_base_mw = 1.0 # MW

E = E_kwh*3600*1000 # julios
P_base = P_base_mw*1e6
H = E/P_base

print(H)
pars = {'H': H,  
        'Phi_f': 1.0,
        'C_eq': 0.1,
        'P_g': 0.1
        }
icdict = {'omega_r': 1.0,
          'v_c': 1.0,
          }

domega_r  = '1.0/(2.0*H)*(-Phi_f*i_q(omega_r))'
dv_c  = '1.0/C_eq*(i_g(v_c)- omega_r*Phi_f*i_q(omega_r)/v_c)'
i_g = 'P_g/v_c'
i_q = 'P_g/(omega_r*Phi_f)'
# auxiliary helper function(s) -- function name: ([func signature], definition)

DSargs = args(name='sys_freq_1')  # struct-like data
DSargs.pars = pars
DSargs.tdata = [0, 900]
DSargs.fnspecs  = {'i_g': (['v_c'], i_g),
                   'i_q': (['omega_r'], i_q),
                  }

DSargs.algparams = {'max_pts': 1000, 'init_step': 0.02, 'stiff': True}

DSargs.varspecs = {'omega_r': domega_r, 
                   'v_c': dv_c, 
                  }

#DSargs.xdomain = {'x': [-2.2, 2.5], 'y': [-2, 2]}
DSargs.ics = icdict

sys = dst.Vode_ODEsystem(DSargs)

900.0


In [45]:
sys.pars['P_g'] = 0.99


traj = sys.compute('test_traj')
pts = traj.sample()


fig, (ax0,ax1,ax2,ax3) = plt.subplots(nrows=4)   # creates a figure with one axe


ax0.plot(pts['t'], pts['omega_r']) 
ax1.plot(pts['t'], pts['v_c']) 


plt.show()




In [12]:

traj = sys.compute('test_traj')

/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/scipy/integrate/_ode.py:869: UserWarning: vode: Excess work done on this call. (Perhaps wrong MF.)
  'Unexpected istate=%s' % istate))


In [50]:
fig, (ax0,ax1,ax2,ax3) = plt.subplots(nrows=4)   # creates a figure with one axe
ax0.plot(pts['t'], pts['Df']) 
ax1.plot(pts['t'], pts['p_g']) 
ax2.plot(pts['t'], pts['p_nc']) 
ax3.plot(pts['t'], pts['e_nc']) 
plt.show()


In [123]:
from IPython.display import display, clear_output
import sys as ssys
from pyswarm import pso

def obj(x):
    sys.pars['K_d'] = x[0]
    sys.pars['K_w1'] = x[1]
    pts = sys.compute('test_traj').sample()
    f_min = np.min(pts['Df'])
    #clear_output(wait=True)
    #print(pts['e_nc'][-1]-9)
    #ssys.stdout.flush()    
    return -f_min

def con(x):
    pts = sys.compute('test_traj').sample()

    e = np.min(pts['e_nc'])-9.5
    clear_output(wait=True)
    print(e)
    ssys.stdout.flush()  
    return e

lb = [-10,0]
ub = [100,200]


xopt, fopt = pso(obj, lb, ub, f_ieqcons=con, args=(), kwargs={},
    swarmsize=10, omega=0.5, phip=0.5, phig=0.5, maxiter=100, minstep=1e-8,
    minfunc=0.1, debug=False)


# Optimum should be around x=[0.5, 0.76] with banana(x)=4.5 and con(x)=0
pts = sys.compute('test_traj').sample()
print('K_d',sys.pars['K_d'])
print('K_w1',sys.pars['K_w1'])
print('e_nc', pts['e_nc'][-1])
print('min Df',np.min(pts['Df']))



0.0737450921186
Stopping search: Swarm best objective change less than 0.1
K_d 100.0
K_w1 98.5635602583
e_nc 9.57374509212
min Df -0.0126599825149


In [124]:
fig, (ax0,ax1,ax2,ax3) = plt.subplots(nrows=4)   # creates a figure with one axe
ax0.plot(pts['t'], pts['Df']) 
ax1.plot(pts['t'], pts['p_g']) 
ax2.plot(pts['t'], pts['p_nc']) 
ax3.plot(pts['t'], pts['e_nc']) 
plt.show()


Exception ignored in: <bound method Vode_ODEsystem.__del__ of Generator sys_freq_1>
Traceback (most recent call last):
  File "/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/PyDSTool/Generator/Vode_ODEsystem.py", line 923, in __del__
    ODEsystem.__del__(self)
  File "/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/PyDSTool/Generator/ODEsystem.py", line 525, in __del__
    ctsGen.__del__(self)
  File "/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/PyDSTool/Generator/baseclasses.py", line 1430, in __del__
    Generator.__del__(self)
  File "/home/jmmauricio/bin/anaconda3/lib/python3.5/site-packages/PyDSTool/Generator/baseclasses.py", line 1381, in __del__
    delattr(eval(finfo[0]), fname)
SyntaxError: unexpected EOF while parsing (<string>, line 0)


In [47]:
import sympy as sym

In [48]:
pars = {'H': H,  
        'Phi_f': 1.0,
        'C_eq': 0.1,
        'P_g': 0.1
        }
x = ['omega_r',
     'v_c']


domega_r  = '1.0/(2.0*H)*(-Phi_f*i_q(omega_r))'
dv_c  = '1.0/C_eq*(i_g(v_c)- omega_r*Phi_f*i_q(omega_r)/v_c)'
i_g = 'P_g/v_c'
i_q = 'P_g/(omega_r*Phi_f)'

In [57]:
omega_r,v_c=sym.symbols(['omega_r','v_c'])

AttributeError: 'list' object has no attribute 'omega_r'